In [1]:
import numpy as np
import pandas as pd
import re
import numpy

from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)
    
pd_atp_matches = pd.read_csv('tennis_atp/atp_matches_2017.csv')
my_data_stats = pd.read_csv('data/match_stats_2017_UNINDEXED.csv')

#select what do I want from the stats dataset
mycolumns=[
           'ms_winner_aces','ms_loser_aces','ms_winner_double_faults','ms_loser_double_faults',
           'ms_winner_first_serves_total','ms_winner_first_serve_points_won','ms_winner_first_serves_in',
           'ms_loser_first_serves_total','ms_loser_first_serve_points_won','ms_loser_first_serves_in',
           'ms_winner_return_points_won','ms_winner_return_points_total',
           'ms_loser_return_points_won','ms_loser_return_points_total',
           'ms_winner_total_points_won','ms_winner_total_points_total',
            'ms_winner_second_serve_points_total','ms_loser_second_serve_points_total']
new_columns=[
             'w_ace','l_ace','w_df','l_df',
             'w_svpt','w_1stWon','w_1stIn',
             'l_svpt','l_1stWon','l_1stIn',   
             'w_ret_w','w_ret_tot',
             'l_ret_w','l_ret_tot',
             'w_pts_w','total_points',
            'w_2ndTot','l_2ndTot']

my_data_stats=my_data_stats.filter(mycolumns, axis=1)
my_data_stats.columns = new_columns

#select what I want from the other dataset
labels=['tourney_id','tourney_name','round','winner_id','winner_name','loser_id','loser_name','score',
        'winner_rank','winner_rank_points','loser_rank','loser_rank_points',
        'w_ace','l_ace','w_df','l_df',
        'w_svpt','w_1stWon','w_1stIn',
        'l_svpt','l_1stWon','l_1stIn',
        'w_2ndWon','l_2ndWon',
        'w_bpSaved','w_bpFaced',
        'l_bpSaved','l_bpFaced']
pd_atp_matches=pd_atp_matches.filter(labels, axis=1)

labels_match=['w_ace','l_ace','w_df','l_df',
                'w_svpt','w_1stWon','w_1stIn',
                'l_svpt','l_1stWon','l_1stIn']
pd_atp_matches=pd.merge(pd_atp_matches,my_data_stats,on=labels_match).reset_index(drop=True)
pd_atp_matches=pd_atp_matches.drop_duplicates(keep='first').reset_index(drop=True)


#calculate number of games
def findOccurrences(s, ch):
    return [i for i, letter in enumerate(s) if letter == ch]

def change_char(s, p, r):
    return s[:p]+r+s[p+1:]

pd_atp_matches=pd_atp_matches.assign(TotGames=0)

#score to number of games
for i in range(0,len(pd_atp_matches.index)):
    str1=pd_atp_matches.loc[i, 'score']
    
    start=findOccurrences(str1, '(')
    end=findOccurrences(str1, ')')

    for k in range(0,len(start)):
        for l in range(0,end[k]-start[k]+1):
            str1=change_char(str1,start[k]+l,' ')

    number_of_games=0
    for j in range (0,len(list(filter(str.isdigit, str1)))):
        number_of_games=number_of_games+int(list(filter(str.isdigit, str1))[j])
    pd_atp_matches.loc[i, 'TotGames'] = number_of_games

    
def SetNan(pandas,column):
    list = pandas.index[(pandas[column].isin([0])) == True].tolist()
    for i in range(0,len(list)):
        pandas.loc[list[i],column]=np.nan
        
SetNan(pd_atp_matches,'w_svpt')
SetNan(pd_atp_matches,'l_svpt')
SetNan(pd_atp_matches,'w_1stIn')
SetNan(pd_atp_matches,'l_1stIn')
SetNan(pd_atp_matches,'w_2ndTot')
SetNan(pd_atp_matches,'l_2ndTot')

In [2]:
predict_dataset = pd.read_csv('tennis_atp/atp_matches_2017.csv')
#select what I want from the other dataset
labels=['winner_id','winner_name','loser_id','loser_name',
        'winner_rank','winner_rank_points','loser_rank','loser_rank_points']
predict_dataset=predict_dataset.filter(labels, axis=1)
display(predict_dataset.loc[467])

winner_id                      106298
winner_name             Lucas Pouille
loser_id                       106421
loser_name            Daniil Medvedev
winner_rank                        17
winner_rank_points               2131
loser_rank                         64
loser_rank_points                 779
Name: 467, dtype: object

In [3]:
#this function returns TMW of a player against a certain adversary

def getMatchesWonPer(myopponent,adversary):
    exp1=pd_atp_matches['winner_id'].isin([myopponent])
    exp2=pd_atp_matches['loser_id'].isin([adversary])
    exp3=pd_atp_matches['loser_id'].isin([myopponent])
    exp4=pd_atp_matches['winner_id'].isin([adversary])

    matches_won=len(pd_atp_matches.loc[(exp1&exp2)].index)
    matches_lost=len(pd_atp_matches.loc[(exp3&exp4)].index)
    if (matches_won+matches_lost)!=0:
        matchesWonPer = matches_won/(matches_won+matches_lost)
    else:
        matchesWonPer=0
        
    return matchesWonPer

predict_dataset=predict_dataset.assign(TMW=0)
for i in range(0,len(predict_dataset.index)):
    TMW1=getMatchesWonPer(predict_dataset.loc[i, 'winner_id'],predict_dataset.loc[i, 'loser_id'])
    predict_dataset.loc[i, 'TMW'] = TMW1 - (1-TMW1)
    

In [4]:
display(predict_dataset.loc[467])

winner_id                      106298
winner_name             Lucas Pouille
loser_id                       106421
loser_name            Daniil Medvedev
winner_rank                        17
winner_rank_points               2131
loser_rank                         64
loser_rank_points                 779
TMW                                 1
Name: 467, dtype: object

In [5]:
all_winner_players=pd_atp_matches.filter(['winner_id','winner_name'], axis=1)
all_winner_players.rename(columns={'winner_id': 'ID','winner_name': 'NAME'}, inplace=True)

all_loser_players=pd_atp_matches.filter(['loser_id','loser_name'], axis=1)
all_loser_players.rename(columns={'loser_id': 'ID','loser_name': 'NAME'}, inplace=True)

#print(all_winner_players)
#print(all_loser_players)

all_players=all_winner_players.append(all_loser_players, ignore_index=True).drop_duplicates()

all_players=all_players.sort_values(by=['ID']).reset_index(drop=True);
#players_list=all_players.ID.unique()
#players_list.sort()
#players_list
#all_players

In [ ]:
#this function returns WRP of a player against a certain adversary

def getSTATSi(myopponent,adversary):
    exp1=pd_atp_matches['winner_id'].isin([myopponent])
    exp2=pd_atp_matches['loser_id'].isin([adversary])
    exp3=pd_atp_matches['loser_id'].isin([myopponent])
    exp4=pd_atp_matches['winner_id'].isin([adversary])

    matches_won=pd_atp_matches.loc[(exp1&exp2)]
    matches_won=matches_won.filter(['tourney_name','winner_id','winner_name','loser_id','loser_name','TotGames',
             'w_ace','l_ace','w_df','l_df',
             'w_svpt','w_1stWon','w_1stIn',
             'l_svpt','l_1stWon','l_1stIn',   
             'w_ret_w','w_ret_tot',
             'l_ret_w','l_ret_tot',
             'w_pts_w','total_points',
             'w_2ndTot','l_2ndTot',
             'w_2ndWon','l_2ndWon',
             'w_bpSaved','w_bpFaced',
             'l_bpSaved','l_bpFaced'], axis=1)
    
    matches_lost=pd_atp_matches.loc[(exp3&exp4)]
    matches_lost=matches_lost.filter(['tourney_name','winner_id','winner_name','loser_id','loser_name','TotGames',
             'w_ace','l_ace','w_df','l_df',
             'w_svpt','w_1stWon','w_1stIn',
             'l_svpt','l_1stWon','l_1stIn',   
             'w_ret_w','w_ret_tot',
             'l_ret_w','l_ret_tot',
             'w_pts_w','total_points',
             'w_2ndTot','l_2ndTot',
             'w_2ndWon','l_2ndWon',
             'w_bpSaved','w_bpFaced',
             'l_bpSaved','l_bpFaced'], axis=1)

    
    #calculate STATS of all the matches against the adversary
    matches_won['FS'] = matches_won['w_1stIn']/matches_won['w_svpt']
    matches_lost['FS'] = matches_lost['l_1stIn']/matches_lost['l_svpt']
    
    matches_won['W1SP'] = matches_won['w_1stWon']/matches_won['w_1stIn']
    matches_lost['W1SP'] = matches_lost['l_1stWon']/matches_lost['l_1stIn']
    
    matches_won['W2SP'] = matches_won['w_2ndWon']/matches_won['w_2ndTot']
    matches_lost['W2SP'] = matches_lost['l_2ndWon']/matches_lost['l_2ndTot']
    
    matches_won['WSP'] = matches_won['W1SP'] * matches_won['FS'] + matches_won['W2SP'] * (1-matches_won['FS'])
    matches_lost['WSP'] = matches_lost['W1SP'] * matches_lost['FS'] + matches_lost['W2SP'] * (1-matches_lost['FS'])

    matches_won['WRP'] = matches_won['w_ret_w']/matches_won['w_ret_tot']
    matches_lost['WRP'] = matches_lost['l_ret_w']/matches_lost['l_ret_tot']
    
    matches_won['TPW'] = matches_won['w_pts_w']/matches_won['total_points']
    matches_lost['TPW'] = (matches_lost['total_points']-matches_won['w_pts_w'])/matches_lost['total_points']
    
    matches_won['ACES'] = (matches_won['w_ace'])/matches_won['TotGames']
    matches_lost['ACES'] = (matches_lost['l_ace'])/matches_lost['TotGames']

    matches_won['DF'] = (matches_won['w_df'])/matches_won['TotGames']
    matches_lost['DF'] = (matches_lost['l_df'])/matches_lost['TotGames']
    
    matches_won['BP'] = matches_won['w_bpSaved']/matches_won['w_bpFaced']
    matches_lost['BP'] = matches_lost['l_bpSaved']/matches_lost['l_bpFaced']

    matches_won['COMPLETE'] = matches_won['WSP']*matches_won['WRP']
    matches_lost['COMPLETE'] = matches_lost['WSP']*matches_lost['WRP']
    
    matches_won=matches_won.append(matches_lost)
    

    
    FS=matches_won['FS'].mean()
    W1SP=matches_won['W1SP'].mean()
    W2SP=matches_won['W2SP'].mean()
    WSP=matches_won['WSP'].mean()
    WRP=matches_won['WRP'].mean()
    TPW=matches_won['TPW'].mean()
    ACES=matches_won['ACES'].mean()
    DF=matches_won['DF'].mean()
    BP=matches_won['BP'].mean()
    COMPLETE=matches_won['COMPLETE'].mean()
    
    
    return (FS,W1SP,W2SP,WSP,WRP,TPW,ACES,DF,BP,COMPLETE)


def getSTATS(id1,id2,label):

    #id1 = 105223 #delpo
    #id2 = 105992 #Ryan harrison

    #id1 = 104229 #nadal
    #id2 = 105032 #groth
    array=[id1,id2]
    locate_1=all_players.loc[(all_players['ID'].isin([array[0]]))]
    locate_2=all_players.loc[(all_players['ID'].isin([array[1]]))]

    if label==1:
        print('El jugador #1 es: ' + locate_1.iloc[0,1])
        print('El jugador #2 es: ' + locate_2.iloc[0,1])

    #print('En el 2017 se enfrentaron a: ' )

    opponents_p1_1=pd_atp_matches.loc[(pd_atp_matches['winner_id'].isin([array[0]]))]
    opponents_p1_1=opponents_p1_1[['loser_id','loser_name']]
    opponents_p1_1.rename(columns={'loser_id': 'ID','loser_name': 'NAME'}, inplace=True)
    opponents_p1_2=pd_atp_matches.loc[(pd_atp_matches['loser_id'].isin([array[0]]))]
    opponents_p1_2=opponents_p1_2[['winner_id','winner_name']]
    opponents_p1_2.rename(columns={'winner_id': 'ID','winner_name': 'NAME'}, inplace=True)
    opponents_p1=opponents_p1_1.append(opponents_p1_2, ignore_index=True).drop_duplicates()

    opponents_p2_1=pd_atp_matches.loc[(pd_atp_matches['winner_id'].isin([array[1]]))]
    opponents_p2_1=opponents_p2_1[['loser_id','loser_name']]
    opponents_p2_1.rename(columns={'loser_id': 'ID','loser_name': 'NAME'}, inplace=True)
    opponents_p2_2=pd_atp_matches.loc[(pd_atp_matches['loser_id'].isin([array[1]]))]
    opponents_p2_2=opponents_p2_2[['winner_id','winner_name']]
    opponents_p2_2.rename(columns={'winner_id': 'ID','winner_name': 'NAME'}, inplace=True)
    opponents_p2=opponents_p2_1.append(opponents_p2_2, ignore_index=True).drop_duplicates()

    common_opponents = pd.merge(opponents_p1, opponents_p2, how='inner', on=['ID']).filter(['ID','NAME_x'], axis=1)
    common_opponents=common_opponents.assign(FS1=0,W1SP1=0,W2SP1=0,WSP1=0,WRP1=0,TPW1=0,ACES1=0,DF1=0,BP1=0,COMPLETE1=0)
    common_opponents=common_opponents.assign(FS2=0,W1SP2=0,W2SP2=0,WSP2=0,WRP2=0,TPW2=0,ACES2=0,DF2=0,BP2=0,COMPLETE2=0)


    for i in range(0,len(common_opponents.index)):
        FS,W1SP,W2SP,WSP,WRP,TPW,ACES,DF,BP,COMPLETE = getSTATSi(id1,common_opponents.loc[i, 'ID'])
        common_opponents.loc[i, 'FS1'] = FS
        common_opponents.loc[i, 'W1SP1'] = W1SP
        common_opponents.loc[i, 'W2SP1'] = W2SP
        common_opponents.loc[i, 'WSP1'] = WSP
        common_opponents.loc[i, 'WRP1'] = WRP
        common_opponents.loc[i, 'TPW1'] = TPW
        common_opponents.loc[i, 'ACES1'] = ACES
        common_opponents.loc[i, 'DF1'] = DF
        common_opponents.loc[i, 'BP1'] = BP
        common_opponents.loc[i, 'COMPLETE1'] = COMPLETE
        
        
        FS,W1SP,W2SP,WSP,WRP,TPW,ACES,DF,BP,COMPLETE = getSTATSi(id2,common_opponents.loc[i, 'ID'])
        common_opponents.loc[i, 'FS2'] = FS
        common_opponents.loc[i, 'W1SP2'] = W1SP
        common_opponents.loc[i, 'W2SP2'] = W2SP
        common_opponents.loc[i, 'WSP2'] = WSP
        common_opponents.loc[i, 'WRP2'] = WRP
        common_opponents.loc[i, 'TPW2'] = TPW
        common_opponents.loc[i, 'ACES2'] = ACES
        common_opponents.loc[i, 'DF2'] = DF
        common_opponents.loc[i, 'BP2'] = BP
        common_opponents.loc[i, 'COMPLETE2'] = COMPLETE
        

    FS=common_opponents['FS1'].mean()-common_opponents['FS1'].mean()
    W1SP=common_opponents['W1SP1'].mean()-common_opponents['W1SP2'].mean()
    W2SP=common_opponents['W2SP1'].mean()-common_opponents['W2SP2'].mean()
    WSP=common_opponents['WSP1'].mean()-common_opponents['WSP2'].mean()
    WRP=common_opponents['WRP1'].mean()-common_opponents['WRP2'].mean()
    TPW=common_opponents['TPW1'].mean()-common_opponents['TPW2'].mean()
    ACES=common_opponents['ACES1'].mean()-common_opponents['ACES2'].mean()
    DF=common_opponents['DF1'].mean()-common_opponents['DF2'].mean()
    BP=common_opponents['BP1'].mean()-common_opponents['BP2'].mean()
    COMPLETE=common_opponents['COMPLETE1'].mean()-common_opponents['COMPLETE2'].mean()
    SERVEADV1=common_opponents['WSP1'].mean()-common_opponents['WRP2'].mean()
    SERVEADV2=common_opponents['WSP2'].mean()-common_opponents['WRP1'].mean()
    SERVEADV=SERVEADV1-SERVEADV2

    
    if label==1:
        display_side_by_side(opponents_p1,opponents_p2,common_opponents)
        
    return(FS,W1SP,W2SP,WSP,WRP,TPW,ACES,DF,BP,COMPLETE,SERVEADV)

##Run to get STATS

predict_dataset=predict_dataset.assign(FS=0,W1SP=0,W2SP=0,WSP=0,WRP=0,TPW=0,ACES=0,DF=0,BP=0,COMPLETE=0,SERVEADV=0)

label=0;
for i in range(0,len(predict_dataset.index)):
#for i in range(0,2):
    if i == 1:
        label=1;
    else:
        label=0;
        
    nFS,nW1SP,nW2SP,nWSP,nWRP,nTPW,nACES,nDF,nBP,nCOMPLETE,nSERVEADV = getSTATS(predict_dataset.loc[i, 'winner_id'],predict_dataset.loc[i, 'loser_id'],label)

    
    predict_dataset.loc[i, 'FS'] = nFS
    predict_dataset.loc[i, 'W1SP'] = nW1SP
    predict_dataset.loc[i, 'W2SP'] = nW2SP
    predict_dataset.loc[i, 'WSP'] = nWSP
    predict_dataset.loc[i, 'WRP'] = nWRP
    predict_dataset.loc[i, 'TPW'] = nTPW
    predict_dataset.loc[i, 'ACES'] = nACES
    predict_dataset.loc[i, 'DF'] = nDF
    predict_dataset.loc[i, 'BP'] = nBP
    predict_dataset.loc[i, 'COMPLETE'] = nCOMPLETE
    predict_dataset.loc[i, 'SERVEADV'] = nSERVEADV

    print('Processing match #' + str(i) + '\n')
    


Processing match #0

El jugador #1 es: Grigor Dimitrov
El jugador #2 es: Milos Raonic


,ID,NAME
0,105453,Kei Nishikori
1,105683,Milos Raonic
2,106233,Dominic Thiem
3,103917,Nicolas Mahut
4,105449,Steve Johnson
5,106331,Christopher Oconnell
6,111202,Hyeon Chung
7,105676,David Goffin
8,105932,Nikoloz Basilashvili
9,104678,Viktor Troicki


Processing match #1

Processing match #2

Processing match #3

Processing match #4

Processing match #5

Processing match #6

Processing match #7

Processing match #8

Processing match #9

Processing match #10

Processing match #11

Processing match #12

Processing match #13

Processing match #14

Processing match #15

Processing match #16

Processing match #17

Processing match #18

Processing match #19

Processing match #20

Processing match #21

Processing match #22

Processing match #23

Processing match #24

Processing match #25

Processing match #26

Processing match #27

Processing match #28

Processing match #29

Processing match #30

Processing match #31

Processing match #32

Processing match #33

Processing match #34

Processing match #35

Processing match #36

Processing match #37

Processing match #38

Processing match #39

Processing match #40

Processing match #41

Processing match #42

Processing match #43

Processing match #44

Processing match #45

Processing match #4

Processing match #362

Processing match #363

Processing match #364

Processing match #365

Processing match #366

Processing match #367

Processing match #368

Processing match #369

Processing match #370

Processing match #371

Processing match #372

Processing match #373

Processing match #374

Processing match #375

Processing match #376

Processing match #377

Processing match #378

Processing match #379

Processing match #380

Processing match #381

Processing match #382

Processing match #383

Processing match #384

Processing match #385

Processing match #386

Processing match #387

Processing match #388

Processing match #389

Processing match #390

Processing match #391

Processing match #392

Processing match #393

Processing match #394

Processing match #395

Processing match #396

Processing match #397

Processing match #398

Processing match #399

Processing match #400

Processing match #401

Processing match #402

Processing match #403

Processing match #404

Processing 

Processing match #719

Processing match #720

Processing match #721

Processing match #722

Processing match #723

Processing match #724

Processing match #725

Processing match #726

Processing match #727

Processing match #728

Processing match #729

Processing match #730

Processing match #731

Processing match #732

Processing match #733

Processing match #734

Processing match #735

Processing match #736

Processing match #737

Processing match #738

Processing match #739

Processing match #740

Processing match #741

Processing match #742

Processing match #743

Processing match #744

Processing match #745

Processing match #746

Processing match #747

Processing match #748

Processing match #749

Processing match #750

Processing match #751

Processing match #752

Processing match #753

Processing match #754

Processing match #755

Processing match #756

Processing match #757

Processing match #758

Processing match #759

Processing match #760

Processing match #761

Processing 

Processing match #1073

Processing match #1074

Processing match #1075

Processing match #1076

Processing match #1077

Processing match #1078

Processing match #1079

Processing match #1080

Processing match #1081

Processing match #1082

Processing match #1083

Processing match #1084

Processing match #1085

Processing match #1086

Processing match #1087

Processing match #1088

Processing match #1089

Processing match #1090

Processing match #1091

Processing match #1092

Processing match #1093

Processing match #1094

Processing match #1095

Processing match #1096

Processing match #1097

Processing match #1098

Processing match #1099

Processing match #1100

Processing match #1101

Processing match #1102

Processing match #1103

Processing match #1104

Processing match #1105

Processing match #1106

Processing match #1107

Processing match #1108

Processing match #1109

Processing match #1110

Processing match #1111

Processing match #1112

Processing match #1113

Processing match

Processing match #1415

Processing match #1416

Processing match #1417

Processing match #1418

Processing match #1419

Processing match #1420

Processing match #1421

Processing match #1422

Processing match #1423

Processing match #1424

Processing match #1425

Processing match #1426

Processing match #1427

Processing match #1428

Processing match #1429

Processing match #1430

Processing match #1431

Processing match #1432

Processing match #1433

Processing match #1434

Processing match #1435

Processing match #1436

Processing match #1437

Processing match #1438

Processing match #1439

Processing match #1440

Processing match #1441

Processing match #1442

Processing match #1443

Processing match #1444

Processing match #1445

Processing match #1446

Processing match #1447

Processing match #1448

Processing match #1449

Processing match #1450

Processing match #1451

Processing match #1452

Processing match #1453

Processing match #1454

Processing match #1455

Processing match

Processing match #1758

Processing match #1759

Processing match #1760

Processing match #1761

Processing match #1762

Processing match #1763

Processing match #1764

Processing match #1765

Processing match #1766

Processing match #1767

Processing match #1768

Processing match #1769

Processing match #1770

Processing match #1771

Processing match #1772

Processing match #1773

Processing match #1774

Processing match #1775

Processing match #1776

Processing match #1777

Processing match #1778

Processing match #1779

Processing match #1780

Processing match #1781

Processing match #1782

Processing match #1783

Processing match #1784

Processing match #1785

Processing match #1786

Processing match #1787

Processing match #1788

Processing match #1789

Processing match #1790

Processing match #1791

Processing match #1792

Processing match #1793

Processing match #1794

Processing match #1795

Processing match #1796

Processing match #1797

Processing match #1798

Processing match

In [ ]:
predict_dataset

In [ ]:
results = predict_dataset.filter(['winner_name','loser_name'], axis=1)

#Ranking Won
results['RANK'] = predict_dataset['winner_rank'] - predict_dataset['loser_rank']
results['POINTS'] = predict_dataset['winner_rank_points'] - predict_dataset['loser_rank_points']

#First Serve In
results['FS'] = predict_dataset['FS']

#Winning in First Serve
results['W1SP'] = predict_dataset['W1SP']

#Winning in Second Serve
results['W2SP'] = predict_dataset['W2SP']

#Winning on Serve Cumulative
results['WSP'] = predict_dataset['WSP']

#Winning in Return
results['TPW'] = predict_dataset['TPW']

results['TMW'] = predict_dataset['TMW']

results['ACES'] = predict_dataset['ACES']

results['DF'] = predict_dataset['DF']

results['BP'] = predict_dataset['BP']

results['COMPLETE'] = predict_dataset['COMPLETE']

results['SERVEADV'] = predict_dataset['SERVEADV']


#results.rename(columns{'winner_id': 'ID1', 'loser_id': 'ID2'}, inplace=True)

results=results.assign(winner=1)

results.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]

In [ ]:
#Now order according to ranking to see how good they are

features = results.filter(['RANK','POINTS','FS','W1SP','W2SP','WSP','WRP','TPW','TMW','ACES','DF','BP','COMPLETE','SERVEADV','winner'], axis=1)

features=features.assign(player1=' ')
features=features.assign(player2=' ')

for i in range(0,len(features.index)):
    if(features.loc[i, 'RANK']<0):
        features.loc[i, 'player1']=results.loc[i,'winner_name']
        features.loc[i, 'player2']=results.loc[i,'loser_name']
    else:
        features.loc[i, 'player1']=results.loc[i,'loser_name']
        features.loc[i, 'player2']=results.loc[i,'winner_name']
        features.loc[i, 'RANK':'SERVEADV']=features.loc[i, 'RANK':'SERVEADV']*-1
        features.loc[i, 'winner']=0

features=features.fillna(0)        
features.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]

features.to_csv('newfeatures2018.csv',header='false',index='false')
features=pd.read_csv('newfeatures2018.csv')
features.loc[[0,1,2,3,4,5,9,1000,1001,1002,1003,1004]]